In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist

# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalize the data
X_train, X_test = X_train / 255.0, X_test / 255.0

# Hyperparameters
input_size = 28
num_classes = 10
filter_size = 3
num_filters_1 = 8
num_filters_2 = 16
pool_size = 2
learning_rate = 0.001
epochs = 10

# Initialize filters and weights
def initialize_filters():
    filters_1 = np.random.randn(num_filters_1, 1, filter_size, filter_size) * 0.1
    filters_2 = np.random.randn(num_filters_2, num_filters_1, filter_size, filter_size) * 0.1
    flatten_size = num_filters_2 * 5 * 5  # 16 * 5 * 5 = 400
    weights_fc = np.random.randn(flatten_size, num_classes) * 0.1
    biases_fc = np.zeros(num_classes)

    return filters_1, filters_2, weights_fc, biases_fc

filters_1, filters_2, weights_fc, biases_fc = initialize_filters()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
def convolution(input, filters):
    num_filters, input_depth, filter_size, _ = filters.shape
    output_dim = input.shape[1] - filter_size + 1
    output = np.zeros((num_filters, output_dim, output_dim))

    for f in range(num_filters):
        for i in range(output_dim):
            for j in range(output_dim):
                region = input[:, i:i+filter_size, j:j+filter_size]
                output[f, i, j] = np.sum(region * filters[f])

    return output

In [ ]:
def relu(feature_map):
    return np.maximum(0, feature_map)

In [ ]:
def max_pooling(input, size=2, stride=2):
    num_filters, height, width = input.shape
    output_dim = (height - size) // stride + 1
    pooled_output = np.zeros((num_filters, output_dim, output_dim))

    for f in range(num_filters):
        for i in range(0, height - size + 1, stride):
            for j in range(0, width - size + 1, stride):
                pooled_output[f, i // stride, j // stride] = np.max(input[f, i:i+size, j:j+size])

    return pooled_output

In [ ]:
def flatten(input):
    return input.reshape(-1)

In [ ]:
def fully_connected(input, weights, biases):
    return np.dot(input, weights) + biases

def softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / np.sum(exp_x)

In [ ]:
def cross_entropy_loss(y_true, y_pred):
    return -np.sum(y_true * np.log(y_pred + 1e-9))

In [ ]:
def forward_propagation(image):
    # Reshape the image to include a channel dimension (1, 28, 28)
    image = image.reshape(1, input_size, input_size)

    # First Convolution + ReLU + Max Pooling
    conv1 = convolution(image, filters_1)
    relu1 = relu(conv1)
    pool1 = max_pooling(relu1)

    # Second Convolution + ReLU + Max Pooling
    conv2 = convolution(pool1, filters_2)
    relu2 = relu(conv2)
    pool2 = max_pooling(relu2)

    # Flatten and Fully Connected Layer
    flat = flatten(pool2)  # This should give a size of 400
    #print(f"Flatten shape: {flat.shape}")  # Should print (400,)
    logits = fully_connected(flat, weights_fc, biases_fc)
    output = softmax(logits)

    return output, conv1, pool1, conv2, pool2, flat

In [ ]:
def update_parameters(output, y_true):
    global weights_fc, biases_fc
    gradient = output - y_true
    weights_fc -= learning_rate * np.outer(flat, gradient)
    biases_fc -= learning_rate * gradient

In [ ]:
for epoch in range(epochs):
    correct_predictions = 0
    total_loss = 0

    for i in range(len(X_train)):
        image = X_train[i]
        label = np.zeros(num_classes)
        label[y_train[i]] = 1
        # Forward pass
        output, conv1, pool1, conv2, pool2, flat = forward_propagation(image)

        # Calculate loss and update parameters
        loss = cross_entropy_loss(label, output)
        total_loss += loss
        update_parameters(output, label)

        # Check accuracy
        if np.argmax(output) == y_train[i]:
            correct_predictions += 1

    accuracy = correct_predictions / len(X_train)
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}, Accuracy: {accuracy:.4f}")

In [ ]:
correct_predictions = 0

for i in range(len(X_test)):
    image = X_test[i]
    label = y_test[i]

    output, _, _, _, _, _ = forward_propagation(image)

    if np.argmax(output) == label:
        correct_predictions += 1

test_accuracy = correct_predictions / len(X_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.8589
